# Multiple arrival processes

In this notebook we will learn how to include multiple arrivals processes in a `simpy` model. We will use exponential distributions here, but any type of distribution, time dependency, of schedule could be included instead.  

We will work with a hypothetical hospital that provides emergency orthopedic surgery to different classes of patient.

![model image](img/multiple_arrivals.png "Urgent care call centre")

| ID | Arrival Type    | Distribution | Mean (mins) |
|----|-----------------|--------------|-------------|
| 1  | Shoulder Trauma | Exponential  | 24.0        |
| 2  | Hip Fracture    | Exponential  | 32.0        |
| 3  | Wrist Fracture  | Exponential  | 21.0        |
| 4  | Ankle Fracture  | Exponential  | 17.0        |



## 1. Imports 

In [1]:
import numpy as np
import itertools
import simpy

## 2. Constants

In [2]:
# default mean inter-arrival times(exp)
IAT_SHOULDER = 24.0
IAT_HIP = 32.0
IAT_WRIST = 21.0
IAT_ANKLE = 17.0

# sampling settings
N_STREAMS = 4
DEFAULT_RND_SET = 0

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables (units = hours)
RUN_LENGTH = 24*10

## 2. Helper classes and functions

In [3]:
class Exponential:
    """
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    """

    def __init__(self, mean, random_seed=None):
        """
        Constructor

        Params:
        ------
        mean: float
            The mean of the exponential distribution

        random_seed: int| SeedSequence, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        """
        self.rand = np.random.default_rng(seed=random_seed)
        self.mean = mean

    def sample(self, size=None):
        """
        Generate a sample from the exponential distribution

        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.

        Returns:
        -------
        float or np.ndarray (if size >=1)
        """
        return self.rand.exponential(self.mean, size=size)

In [4]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Experiment class

In [5]:
class Experiment:
    """
    Encapsulates the concept of an experiment 🧪 for the Orthopedic Surgey
    trauma arrival simulator. Manages parameters, PRNG streams and results.
    """

    def __init__(
        self,
        random_number_set=DEFAULT_RND_SET,
        n_streams=N_STREAMS,
        iat_shoulder=IAT_SHOULDER,
        iat_hip=IAT_HIP,
        iat_wrist=IAT_WRIST,
        iat_ankle=IAT_ANKLE,
    ):
        """
        The init method sets up our defaults.
        """
        # sampling
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # store parameters for the run of the model
        self.iat_shoulder = iat_shoulder
        self.iat_hip = iat_hip
        self.iat_wrist = iat_wrist
        self.iat_ankle = iat_ankle

        # initialise results to zero
        self.init_results_variables()

        # initialise sampling objects
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of pseudo random numbers used by
            the distributions in the simulation.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Create the distributions used by the model and initialise
        the random seeds of each.
        """
        # produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # create distributions

        # inter-arrival time distributions
        self.arrival_shoulder = Exponential(
            self.iat_shoulder, random_seed=self.seeds[0]
        )

        self.arrival_hip = Exponential(
            self.iat_hip, random_seed=self.seeds[0]
        )

        self.arrival_wrist = Exponential(
            self.iat_wrist, random_seed=self.seeds[0]
        )

        self.arrival_ankle = Exponential(
            self.iat_ankle, random_seed=self.seeds[0]
        )
        

    def init_results_variables(self):
        """
        Initialise all of the experiment variables used in results
        collection.  This method is called at the start of each run
        of the model
        """
        # variable used to store results of experiment
        self.results = {}
        self.results["n_shoulders"] = 0
        self.results["n_hips"] = 0
        self.results["n_wrists"] = 0
        self.results["n_ankles"] = 0

## 4. A function per arrival source

The first approach we will use is creating an arrival generator per source.  There will be some code redundancy, but it will a clear design for others to understand.

In [6]:
def shoulder_arrivals_generator(env, args):
    """
    Arrival process for shoulders.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_shoulder.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_shoulders"] = patient_count
        
        trace(f"{env.now:.2f}: SHOULDER arrival.")

In [7]:
def hip_arrivals_generator(env, args):
    """
    Arrival process for hips.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_hip.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_hips"] = patient_count
        trace(f"{env.now:.2f}: HIP arrival.")

In [8]:
def wrist_arrivals_generator(env, args):
    """
    Arrival process for wrists.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_wrist.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_wrists"] = patient_count
        trace(f"{env.now:.2f}: WRIST arrival.")

In [9]:
def ankle_arrivals_generator(env, args):
    """
    Arrival process for ankles.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_wrist.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_ankles"] = patient_count
        trace(f"{env.now:.2f}: ANKLE arrival.")

## 5. Single run function

In [10]:
def single_run(
    experiment, 
    rep=0,
    run_length=RUN_LENGTH
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    rc_period: float, optional (default=RUN_LENGTH)
        The run length of the model
    """

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # we pass all arrival generators to simpy 
    env.process(shoulder_arrivals_generator(env, experiment))
    env.process(hip_arrivals_generator(env, experiment))
    env.process(wrist_arrivals_generator(env, experiment))
    env.process(ankle_arrivals_generator(env, experiment))

    # run for warm-up + results collection period
    env.run(until=run_length)

    # return the count of the arrivals
    return experiment.results

In [11]:
TRACE = True
experiment = Experiment()
results = single_run(experiment)
results

16.03: ANKLE arrival.
42.33: ANKLE arrival.
47.07: ANKLE arrival.
51.71: ANKLE arrival.
69.16: WRIST arrival.
70.55: WRIST arrival.
79.04: SHOULDER arrival.
89.46: WRIST arrival.
89.50: ANKLE arrival.
95.66: WRIST arrival.
97.36: SHOULDER arrival.
105.39: HIP arrival.
127.42: SHOULDER arrival.
129.81: HIP arrival.
132.84: SHOULDER arrival.
134.69: WRIST arrival.
135.00: WRIST arrival.
137.80: ANKLE arrival.
138.14: SHOULDER arrival.
154.52: ANKLE arrival.
161.86: ANKLE arrival.
169.89: HIP arrival.
177.12: HIP arrival.
181.34: SHOULDER arrival.
182.92: SHOULDER arrival.
184.18: HIP arrival.
203.25: ANKLE arrival.
204.53: SHOULDER arrival.
211.62: SHOULDER arrival.
218.83: ANKLE arrival.
218.96: ANKLE arrival.
236.34: ANKLE arrival.


{'n_shoulders': 9, 'n_hips': 5, 'n_wrists': 6, 'n_ankles': 12}

## A hospital that only provides surgery for hip fractures

In [12]:
M = 1_000_000
experiment = Experiment(iat_shoulder=M, iat_wrist=M, iat_ankle=M)
results = single_run(experiment)
results

105.39: HIP arrival.
129.81: HIP arrival.
169.89: HIP arrival.
177.12: HIP arrival.
184.18: HIP arrival.


{'n_shoulders': 0, 'n_hips': 5, 'n_wrists': 0, 'n_ankles': 0}

## A single arrival generator function

We can write less code by modifying the `Experiment` class to use `dict` to store the distributions and by creating a generator function that accepts parameters.  

The code is more flexible, at the cost of readability (and potential to make more mistakes) for some less experienced coders.

In [13]:
class Experiment:
    """
    Encapsulates the concept of an experiment 🧪 for the Orthopedic Surgey
    trauma arrival simulator. Manages parameters, PRNG streams and results.
    """

    def __init__(
        self,
        random_number_set=DEFAULT_RND_SET,
        n_streams=N_STREAMS,
        iat_shoulder=IAT_SHOULDER,
        iat_hip=IAT_HIP,
        iat_wrist=IAT_WRIST,
        iat_ankle=IAT_ANKLE,
    ):
        """
        The init method sets up our defaults.
        """
        # sampling
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # store parameters for the run of the model
        self.iat_shoulder = iat_shoulder
        self.iat_hip = iat_hip
        self.iat_wrist = iat_wrist
        self.iat_ankle = iat_ankle
        
        # we will store all code in distributions
        self.dists = {}

        # initialise results to zero
        self.init_results_variables()

        # initialise sampling objects
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of pseudo random numbers used by
            the distributions in the simulation.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Create the distributions used by the model and initialise
        the random seeds of each.
        """
        # produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # create distributions
        
        # inter-arrival time distributions
        self.dists["shoulder"] = Exponential(
            self.iat_shoulder, random_seed=self.seeds[0]
        )
        
        self.dists["hip"] = Exponential(
            self.iat_hip, random_seed=self.seeds[1]
        )

        self.dists["wrist"] = Exponential(
            self.iat_wrist, random_seed=self.seeds[2]
        )

        self.dists["ankle"] = Exponential(
            self.iat_ankle, random_seed=self.seeds[3]
        )
        

    def init_results_variables(self):
        """
        Initialise all of the experiment variables used in results
        collection.  This method is called at the start of each run
        of the model
        """
        # variable used to store results of experiment
        self.results = {}
        self.results["n_shoulders"] = 0
        self.results["n_hips"] = 0
        self.results["n_wrists"] = 0
        self.results["n_ankles"] = 0

In [14]:
def trauma_generator(env, trauma_type, args):
    """
    Modified generator for arrivals.
    Now works across all trauma types.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    trauma_type: str
        string representing the type of patient e.g. "shoulder"

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.dists[trauma_type].sample()
        yield env.timeout(inter_arrival_time)
     
        args.results[f"n_{trauma_type}s"] = patient_count
        trace(f"{env.now:.2f}: {trauma_type.upper()} arrival.")

In [15]:
def single_run(
    experiment, 
    rep=0,
    run_length=RUN_LENGTH
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    rc_period: float, optional (default=RUN_LENGTH)
        The run length of the model
    """

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # we pass all arrival generators to simpy 
    for trauma_type in experiment.dists.keys():
        env.process(trauma_generator(env, trauma_type, experiment))

    # run for warm-up + results collection period
    env.run(until=run_length)

    # return the count of the arrivals
    return experiment.results

In [16]:
TRACE = False
experiment = Experiment()
results = single_run(experiment)
results

{'n_shoulders': 9, 'n_hips': 6, 'n_wrists': 14, 'n_ankles': 14}

In [17]:
M = 1_000_000
experiment = Experiment(iat_shoulder=M, iat_wrist=M, iat_ankle=M)
results = single_run(experiment)
results

{'n_shoulders': 0, 'n_hips': 6, 'n_wrists': 0, 'n_ankles': 0}